In [1]:
import numpy as np
import seaborn as sn
import pandas as pd

read file

In [3]:
data=pd.read_csv('медицинские_протоколы.csv',sep=';')

In [5]:
list(data.columns.values)

['MedicalRecordKey',
 'MedicalRecordDate',
 'PatientKey',
 'Возраст',
 'Пол',
 'СпециальностьВрача',
 'Жалобы',
 'ПеренесенныеЗаболевания',
 'ПеренесенныеОперации',
 'ПринимаемыеПрепараты']

In [6]:
complaints=data['Жалобы']
complaints

0                   на мажущие  выделения из половых путей
1        на дискомфорт при мочеиспускании на учащенное ...
2                  На покраснение и шелушение кожи головы.
3        Подписано информированное добровольное согласи...
4        Жалобы на болезненность и нагрубание правой мо...
                               ...                        
12760               покраснение отек 1 пальца правой стопы
12761    на момент осмотра активных жалоб по ЛОР не пре...
12762    НАЛ  жалобы на возникновение болевых ощущенив ...
12763    Нестабильность цифр АД, на повышение цифр АД д...
12764     изжога после приема пищи , переполнение в эпи...
Name: Жалобы, Length: 12765, dtype: object

In [48]:
def gen_word(cmp,ind):
    new_cmp=""
    while(ind<len(cmp) and (ord(cmp[ind])==32 or ord(cmp[ind])==46)):
        ind+=1
    while (ind<len(cmp) and (ord(cmp[ind])!=32)):
        if(ord(cmp[ind])>=97 and ord(cmp[ind])<=122):
            new_cmp+=chr(ord(cmp[ind])-32)
        elif(ord(cmp[ind])>=1040 and ord(cmp[ind])<=1071):
            new_cmp+=chr(ord(cmp[ind])+32)
        elif((ord(cmp[ind])==92 or cmp[ind]=="/") and len(new_cmp)>0 and new_cmp[-1]=="/"):
            pass
        elif(ord(cmp[ind])==92):
            new_cmp+="/"
        else:
            new_cmp+=cmp[ind]
        ind+=1
    return new_cmp,ind
new_complaints=[]
for cmplnt in complaints:
    new_cmplnt=[]
    cmplnt = cmplnt.split(",")
    for cmp in cmplnt:
        new_cmp=""
        s=0
        word=""
        while(s<len(cmp)):
            word=""
            word,s=gen_word(cmp,s)
            new_cmp+=word+" "
        while(len(new_cmp)>0 and (new_cmp[-1]==" " or new_cmp[-1]==".")):
            new_cmp=new_cmp[:-1:]
        new_cmplnt.append(new_cmp)
    new_complaints.append(new_cmplnt)
new_complaints

[['на мажущие выделения из половых путей'],
 ['на дискомфорт при мочеиспускании на учащенное мочеиспускание'],
 ['на покраснение и шелушение кожи головы'],
 ['подписано информированное добровольное согласие на получение первичной медико-санитарной помощи. прием кардиолога повторно жалобы на фоне стресса повышение ад 145/90 ммрттс с головными болями 2 балла по ваш',
  'на эпизоды учащенного сердцебиения без четкой связи с физической нагрузкой',
  'перебои в работе сердца -'],
 ['жалобы на болезненность и нагрубание правой молочной железы'],
 ['на боли в левом коленном суставе'],
 ['на повышение ад до 150/100 мм рт ст', ''],
 ['жалобы на отрыжку',
  'периодическую тяжесть в животе-эпигастральной области',
  'правом подреберье',
  'после еды',
  'периодическое вздутие живота',
  'дискомфорт при дефекации'],
 ['жалобы на частые головные боли',
  'ноющие',
  'по ваш на 5-6б',
  'чаще в затылочной области',
  'до тошноты',
  'двоение в глазах',
  'появление молний',
  'лучей и вспышек',
  'ч

In [131]:
t=[]
dict_synonym={'нет':'жалоб нет', 'не предъявляет':'жалоб нет', 
              'на момент осмотра жалоб нет':'жалоб нет','на момент осмотра жалоб не предъявляет':'жалоб нет', 
              'жалоб на момент осмотра не предъявляет':'жалоб нет', 'общая слабость':'слабость','утомляемость':'слабость',
              'общая сонливость':'сонливость', 'жалоб в момент осмотра не предъявляет':'жалоб нет','общую слабость':'слабость','повышенную утомляемость':'утомляемость','общую слабость':'слабость', 'на общую слабость':'слабость','на дискомфорт':'дискомфорт','дискомфорт в горле':'боль в горле'}
for i in new_complaints:
    for j in i:
        t.append(j)

types_of_complaints=dict()
for i in t:
    if(i in dict_synonym.keys() and dict_synonym[i] in types_of_complaints.keys()):
        types_of_complaints[dict_synonym[i]]+=1
    elif(i in types_of_complaints.keys()):
        types_of_complaints[i]+=1
    else:
        types_of_complaints[i]=1
types_of_complaints.pop('')


173

In [132]:
def to_dict(s):
    d = dict()
    for i in s:
        if i in d.keys():
            d[i] += 1
        else:
            d[i] = 1
    return d

def collide(a, b):
    a = to_dict(a)
    res = 0
    b = to_dict(b)
    for i in set(list(a.keys()) + list(b.keys())):
        res += min(a.get(i, 0), b.get(i, 0)) / max((a.get(i, 0), b.get(i, 0)))
    
    return res / (len(set(list(a.keys()) + list(b.keys()))))

i=0;j=0
while i < len(types_of_complaints.keys()):
    while j < len(types_of_complaints.keys()):
        if i!=j:
            #print(list(types_of_complaints.keys())[i],list(types_of_complaints.keys())[j],collide(list(types_of_complaints.keys())[i],list(types_of_complaints.keys())[j]))
            if collide(list(types_of_complaints.keys())[i],list(types_of_complaints.keys())[j])>=0.75:
                types_of_complaints[list(types_of_complaints.keys())[i]]+=types_of_complaints[list(types_of_complaints.keys())[j]]
                types_of_complaints.pop(list(types_of_complaints.keys())[j])
                #print(list(types_of_complaints.keys())[i],list(types_of_complaints.keys())[j])
                i-=1
                j-=1
        j+=1
    i+=1
                
                
            

In [133]:
sorted_types_of_complaints=sorted(types_of_complaints.items(),key=lambda x:x[1],reverse=True)